In [1]:
%matplotlib notebook

from torch import load
import torch
import numpy as np
from schnetpack.data import ASEAtomsData as AtomsData
import schnetpack as spk
import ase.db
import ase.io
from ase.io import read,write
import joblib
import lightgbm as lgb
from openbabel import openbabel
from rdkit import Chem
import matplotlib.pyplot as plt
import os
from qcnico.plt_utils import histogram




INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:rdkit:Enabling RDKit 2024.03.6 jupyter extensions


In [2]:
def load_model(target,path):
    if target!='PCE':
        model1=load(path, map_location=torch.device('cpu'))
        model1.eval()
        return model1
    else:
        model= joblib.load(path)
        
        return model
    

In [3]:
from rdkit.Chem import rdDetermineBonds

def ase_to_rdkit(atoms):
    """
    Convert an ASE Atoms object to an RDKit Mol object.
    
    Args:
        atoms (ase.Atoms): The input ASE Atoms object.
    
    Returns:
        Chem.Mol: The corresponding RDKit Mol object.
    """
    # Convert ASE Atoms to XYZ string
    print('[ase_to_rdkit] Start...',flush=True)
    xyz_str = f"{len(atoms)}\n\n"
    for symbol, position in zip(atoms.get_chemical_symbols(), atoms.positions):
        xyz_str += f"{symbol} {position[0]:.6f} {position[1]:.6f} {position[2]:.6f}\n"
    
    print('[ase_to_rdkit] xyz string acquired',flush=True)
    # Convert XYZ to RDKit Mol
    raw_mol = Chem.MolFromXYZBlock(xyz_str)
    mol = Chem.Mol(raw_mol)
    print('[ase_to_rdkit] mol object constructed',flush=True)
    mol.UpdatePropertyCache() # this calculates implicit valence of each atom and prevents this from crashing
    # mol = Chem.AddHs(mol)
    # print('[ase_to_rdkit] added Hs',flush=True)

    if mol is None:
        raise ValueError("RDKit failed to create a Mol object from XYZ data.")
        # Infer bonds based on atomic distances
    rdDetermineBonds.DetermineBonds(mol)
    print('[ase_to_rdkit] End!',flush=True)
    return mol


    

def cal_nd(mol):
    print('Starting cal_nd...',flush=True)
    atoms=mol.toatoms()
    # print(atoms)
    mol = ase_to_rdkit(atoms)
    # print(mol)

    # print(atoms)
    # write('mol.xyz',atoms)
    # obConversion = openbabel.OBConversion()
    # obConversion.SetInAndOutFormats("xyz", "mol")
    # mol = openbabel.OBMol()
    # obConversion.ReadFile(mol, "mol.xyz")   # Open Babel will     uncompress automatically
    # mol.AddHydrogens() 
    # obConversion.WriteFile(mol, '1.mol')

    #calculate Nd         
    # mol = Chem.MolFromMolFile('1.mol')        
    n = len(mol.GetAtoms())         
    Nd = 0        
    for i in range(0,n):
        atom = mol.GetAtomWithIdx(i)
        #判断原子是否为芳香性
        if atom.GetIsAromatic() == True:
            Nd += 1
        if atom.GetIsAromatic() == False:
            #判断原子价电子是否等于总饱和度
            if atom.GetTotalValence() != atom.GetTotalDegree():
                Nd += 1
            if atom.GetTotalValence() == atom.GetTotalDegree():
                #判断原子是否在环上
                if atom.IsInRing() == True:
                    Nd += 1
    print('Nd = ', Nd, flush=True) 
    return Nd                                                                     

In [4]:
def cal_prop(moln,molo,tag):
    
    
    al=.0

    if molo.data.Acceptor=='PC61BM':
        al= -3.70
        adl= 0.077824564
    elif molo.data.Acceptor=='PC71BM':
        al= -3.91
        adl= 0.033470005
    if tag=='edahl':
        prop=al-float(molo.data.HOMO)
    if tag=='edall':
        prop=float(molo.data.LUMO)-al
    if tag=='adlumo':
        prop=adl
    if tag=='nd':
        # prop=cal_nd(moln)
        prop = float(molo.data.ND)


    return prop

In [5]:
def pred_data( model,tag,data):
     
            
    if tag== 'PCE':
        return pred_pce(model,data)
        
    else :
         return pred_prop(model,tag,data)    
             

In [6]:
def pred_pce(model,data):
    db=ase.db.connect(data)
    pce=[]
    ids=[]
    for row in db.select():
        x=[]
        x.extend((row.data.homo,row.data.lumo,row.data.edahl,row.data.edall,row.data.et1,row.data.nd,row.data.adlumo,row.data.dhomo,row.data.dlumo))
        y = model.predict(np.array(x).reshape(1,-1)).tolist()
        print(f'Device {id} ---> PCE = ', y, flush=True)
#         print(y)
        pce.extend(y)
        ids.append(row.id)
        
    return ids,pce

In [7]:
def pred_prop(model,tag,data):
    pred=AtomsData(data)
    pred_loader = spk.AtomsLoader(pred, batch_size=10) #40!!
    
    for count, batch in enumerate(pred_loader): 
        datapred = model(batch)
        ids=batch['_idx'].numpy().tolist()
        datapred=datapred[tag].detach().numpy().tolist()
        yield datapred,ids

In [8]:
def copy_prop(db,tag):
    for row in db.select():
        id=row.id
        prop = float(row.data[tag])
        yield prop, id

In [9]:
def write_results(predata,db):
    
    for num in predata.keys():
        print(f'{num}',end=' --> ',flush=True)
        print(predata[num],flush=True)
        db.update(id=num,data=predata[num])
        # for prop in predata[num].keys():
            
            # db.update(id=num+1, **{prop: predata[num][prop]}) 
    return 0

In [10]:
from data_utils import split_train_test

def train_pce_model(model,Xtrain, ytrain):
        
    # print(type(model_params))
    # print(model_  params)
    # train_data = lgb.Dataset(Xtrain,label=ytrain)

    # model = lgb.train(model_params, train_data, num_boost_round=100)
    print(model.get_params())
    model.fit(Xtrain, ytrain)

    return model

In [11]:
def write_train_db():
    target=['et1','dhomo','dlumo','homo','lumo','pce'] # need to predict with schnet; i.e. need SLI-GNN to get these ppties
    otarget=['ET1','DH','DL','HOMO','LUMO','PCE'] # need to predict with schnet; i.e. need SLI-GNN to get these ppties
    target2=['nd','edahl','edall','adlumo'] # no need to predict; can compute these ppties from the above targets and by knowing the acceptor
    # final_target = ['pce']
    all_targets = target + target2 #all of the target ppties used by LightGBM to predict PCE

    output_db_path = 'data/train2.db'
    if os.path.exists(output_db_path):
         os.remove(output_db_path)

    db=ase.db.connect(output_db_path) #output database where results get written to 
    odb=ase.db.connect('data/train.db') #input database containing all of the structures of the starting compounds

    nmols = odb.count()
    predata = {(n+1):{tag:None for tag in all_targets} for n in range(nmols)} # dict of dicts, where the 'outer' dict keys are acceptor IDs and the values are dicts containing their molecular ppties (and the acceptor type)
    print(predata,flush=True)

    # Copy structures from input db to output db; then only work with output db
    for mol in odb.select():      
            atom=mol.toatoms()
            db.write(atom)

    for tag, otag in zip(target, otarget):
        for property,id in copy_prop(odb,otag):
            print(id, property, flush=True)
            # predata.update({id:{tag:property}}) #add property:value pair to each molecule one property at a time (sprop is a singleton)
            predata[id][tag] = property 
    print(predata,flush=True)
    # write_results(predata,tag,db)    
#         print(predata)
    for tag in target2:
        for moln,molo in zip(db.select(),odb.select()): #molecules in both databases are ordered in the same way, so `molo` and `moln` refer to the same molecule; need to `molo`` (from input db `odb`) because it contains info about the acceptor
            sprop=cal_prop(moln,molo,tag)
            print(f'\nmoln.id = {moln.id}\tmolo.id = {molo.id}',flush=True)    
            # sid=moln.id-1 # for zero indexing?
            sid=moln.id
            # predata.update({sid:{tag:sprop}})
            predata[sid][tag] = sprop
           
    write_results(predata,db) #save molecules and their properties to a '.db' file
    return db


def inspect_feature(x,feature_name):
    fig, ax = plt.subplots()
    print(feature_name, end = ' --> ')
    fig, ax = histogram(x,bins=50,plt_objs=(fig,ax),xlabel=feature_name,show=False,usetex=False)
    plt.show()
    print(f'mean = {np.mean(x)} ; std = {np.std(x)}')




def test_model_from_db(Xtrain, ytrain, Xtest, ytest, model_type='best so far'):
    # Xtrain, ytrain, Xtest, ytest = split_train_test(db)
    # model = joblib.load(model_params_path)
    # model.set_params(objective='regression', metric='rmse')

    if model_type == 'overfit':
        model = lgb.LGBMRegressor(
            learning_rate=0.5,       # Even more aggressive learning
            max_depth=-1,            # No limit on depth
            num_leaves=2**16,        # A massive number of leaves to force splits
            min_child_samples=1,     # Allow splitting until only 1 sample remains
            n_estimators=5000,       # Huge number of trees to ensure memorization
            reg_alpha=0,             # No L1 regularization
            reg_lambda=0,            # No L2 regularization
            subsample=1.0,           # Use 100% of the data
            colsample_bytree=1.0,    # Use 100% of features
            min_split_gain=0,        # Allow splits even with **no gain**
            importance_type='gain',  # Prioritize gain-based splitting
            force_col_wise=True,     # Helps force more splits
            extra_trees=True,        # Randomizes split selection to allow more splits
            deterministic=True,      # Ensures no randomness
            boosting_type='gbdt',    # Standard boosting
            objective='regression',
            metric='rmse',
            random_state=42
        )

    elif model_type == 'paper':
        model = lgb.LGBMRegressor(
        learning_rate=0.15, 
        max_depth=9,
        # min_child_samples=5,
        n_estimators=39,
        num_leaves = 35,
        random_state=399,
        # reg_alpha=0.2,
        # reg_lambda=0.01,
        objective='regression',
        metric='rmse')

    else:
        model = lgb.LGBMRegressor(
        learning_rate=0.05, 
        max_depth=-1,
        min_child_samples=5,
        n_estimators=500,
        num_leaves = 2**12,
        random_state=399,
        reg_alpha=0.2,
        reg_lambda=0.01,
        objective='regression',
        metric='rmse')
    model = train_pce_model(model,Xtrain,ytrain)

    model_file = 'weights/lgb_nico'

    print('Saving model to: ', model_file)
    # model.booster_.save_model(model_file)
    joblib.dump(model, model_file)


    ytest_pred = model.predict(Xtest) 

    print(f'********** r =  {np.corrcoef(ytest, ytest_pred)[0,1]} **********')

    yrange = np.linspace(np.min(ytrain),np.max(ytrain),100)

    fig, ax = plt.subplots()
    ax.scatter(ytest,ytest_pred)
    ax.plot(yrange,yrange,'k--',lw=0.7)
    ax.set_ylabel('Predicted PCE (%)')
    ax.set_xlabel('PCE (%)')
    plt.savefig('lgb_perf_test.png',dpi=300)
    plt.show()

    return model

In [12]:
# db = write_train_db()
db = ase.db.connect("data/train2.db")
Xtrain, ytrain, Xtest, ytest, feature_names = split_train_test(db,normalize=True,minmax_scale=False)

train_data = np.concatenate((Xtrain, ytrain.reshape(-1,1)),axis=1).T
# feature_names = db.get(id=1).data.keys()
for x, ft in zip(train_data, feature_names):
    inspect_feature(x,ft)


model = test_model_from_db(Xtrain[:,:-1],ytrain,Xtest[:,:-1],ytest,model_type='paper') #exclude adlumo (only 2 unique values)

<IPython.core.display.Javascript object>

homo --> [plt_utils.histogram] dx = 0.16401595078426376
mean = 0.037555851250540354 ; std = 0.8965041619495204


<IPython.core.display.Javascript object>

lumo --> [plt_utils.histogram] dx = 0.17516715582382325
mean = 0.029142061946652576 ; std = 0.947875536558185


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

adlumo --> [plt_utils.histogram] dx = 0.04305557891470846
mean = 0.014800355251918054 ; std = 1.0057682884041677


<IPython.core.display.Javascript object>

pce --> [plt_utils.histogram] dx = 0.19599999999999998
mean = 4.696656249999999 ; std = 1.9935390344399926
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.15, 'max_depth': 9, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 39, 'n_jobs': None, 'num_leaves': 35, 'objective': 'regression', 'random_state': 399, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'rmse'}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 771
[LightGBM] [Info] Number of data points in the train set: 320, number of used features: 8
[LightGBM] [Info] Start training from score 4.696656
[LightGBM] [Warning] No further splits with positiv

<IPython.core.display.Javascript object>

In [13]:
feature_importance = model.feature_importances_
# feature_importance = model.booster_.feature_importance_(importance_type="gain")
plt.bar(range(len(feature_importance)), feature_importance)
plt.xlabel("Feature Name")
plt.ylabel("Importance Score")
plt.xticks(np.arange(len(feature_importance)), feature_names[:8], rotation=30)
plt.savefig("feature_importances",dpi=300,bbox_inches='tight')
plt.show()

<IPython.core.display.Javascript object>

In [14]:
fig, ax = plt.subplots()
ax = lgb.plot_importance(model.booster_,ax=ax,importance_type="gain",)
# ax.set_xticks(np.arange(len(feature_importance)), feature_names[:8], rotation=30)
plt.savefig("feature_importances_gain",dpi=300,bbox_inches='tight')
plt.show()


<IPython.core.display.Javascript object>

In [ ]:
from qcnico.plt_utils import multiple_histograms
from data_utils import make_indoor_data
from matplotlib import rcParams


device_xlsx = '/Users/nico/Desktop/scripts/OPVGCN/data/data-indoor.xlsx'
donor_csv = '/Users/nico/Desktop/scripts/OPVGCN/data/donors_emna_dft.csv'
acceptor_csv = '/Users/nico/Desktop/scripts/OPVGCN/data/acceptors_emna_dft.csv'

Xindoor, yindoor = make_indoor_data(device_xlsx, donor_csv, acceptor_csv, normalize=True)

ypred_indoor = model.predict(Xindoor)

yrange = np.linspace(np.min(yindoor), np.max(yindoor),100)

y_train_min = np.min(ytrain)
y_train_max = np.max(ytrain)

#Look only at data points with PCEs similar to training data
pce_filter = (yindoor >= y_train_min) * (yindoor <= y_train_max)
print(f'*** PCE filter = {pce_filter.sum()} ***')
# Xindoor = Xindoor[pce_filter]
# yindoor = yindoor[pce_filter]

rcParams['text.usetex'] = False
print(rcParams['text.usetex'])


fig,ax = plt.subplots()
ax.scatter(yindoor,ypred_indoor)
ax.plot(yrange,yrange,'k--',lw=0.7)
ax.set_xlabel('PCE (%)')
ax.set_ylabel('Predicted PCE (%)')
plt.savefig('lgb_perf_indoor.png',dpi=300,bbox_inches="tight")
plt.show()

print(f'********** r =  {np.corrcoef(yindoor, ypred_indoor)[0,1]} **********')

num_features = 8

for k in range(num_features):
    fig, ax = plt.subplots()
    rcParams['text.usetex'] = False
    # plt_dat = [Xtrain[:,k],Xindoor[:,k]]
    plt_dat = [Xtrain[:,k],Xindoor[pce_filter,k]]
    fig, ax = multiple_histograms(plt_dat,labels=['train', 'indoor'],bins=50,xlabel=feature_names[k],plt_objs=(fig,ax),normalised=True,show=False,usetex=False)
    ax.legend()
    plt.savefig(f'compare_{feature_names[k]}_train_v_indoor_smallPCE.png',dpi=300,bbox_inches="tight")
    plt.show()

fig, ax = plt.subplots()
fig,ax = multiple_histograms([ytrain, yindoor],labels=['train', 'indoor'],bins=50,xlabel='PCE',plt_objs=(fig,ax),normalised=True,show=False,usetex=False)
plt.savefig(f'compare_PCE_train_v_indoor.png',dpi=300)
plt.show()



Already saw  ('PCDTBT', 'PC71BM')
Acceptors BTA3 is missing from Excel sheet
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('PTB7-Th(PCE10)', 'PC71BM')
Already saw  ('DTCPB', 'C70')
Already saw  ('P1', 'PC71BM')
Already saw  ('P3HT', 'PC61BM')
Already saw  ('P3HT', 'PC61BM')
Already saw  ('P3HT', 'ICBA')
Already saw  ('PPDT2FBT', 'ITIC-M')
Already saw  ('PPDT2FBT', 'ITIC-F')
Already saw  ('PPDT2FBT', 'tPDI2N-EH')
Already saw  ('PPDT2FBT', 'PC61BM')
Acceptors ITIC is missing from Excel sheet
Already saw  ('CD1', 'ITIC')
Already saw  ('PBDB-TF(PM6)', 'IT-4F')
Already saw  ('PPDT2FBT', 'ITIC-M')
Already saw  

<IPython.core.display.Javascript object>

********** r =  0.3507918167288157 **********


<IPython.core.display.Javascript object>

[plt_utils.histogram] dx = 0.16401595078426376
[plt_utils.histogram] dx = 0.051654082161820415


...

[plt_utils.histogram] dx = 0.16528123396482206
[plt_utils.histogram] dx = 0.07447968106711338


<IPython.core.display.Javascript object>

[plt_utils.histogram] dx = 0.1573695994006189
[plt_utils.histogram] dx = 0.08469157791467281


<IPython.core.display.Javascript object>

[plt_utils.histogram] dx = 0.19599999999999998
[plt_utils.histogram] dx = 0.534


In [16]:
yrange = np.linspace(0,10, 100)

fig, ax = plt.subplots()
ax.scatter(yindoor,ypred_indoor)
ax.set_xlabel('PCE (%)')
ax.set_ylabel('Predicted PCE (%)')
ax.plot(yrange,yrange,'k--',lw=0.7)
ax.set_ylim([np.min(ytrain), np.max(ytrain)])
ax.set_xlim([np.min(ytrain), np.max(ytrain)])
plt.savefig('lgb_perf_indoor_smallPCE.png',dpi=300,bbox_inches="tight")
plt.show()


<IPython.core.display.Javascript object>

In [17]:

from qcnico.plt_utils import multiple_histograms
from data_utils import make_cc_data

db_path = 'data/CADAC.db'

Xcadac, ycadac = make_cc_data(db_path, normalize=True,minmax_scale=True)

ypred_cadac = model.predict(Xcadac)

plt.scatter(ycadac,ypred_cadac)
plt.xlabel('PCE (\%)')
plt.ylabel('Predicted PCE (\%)')
plt.show()

num_features = 8

for k in range(num_features):
    fig, ax = plt.subplots()
    plt_dat = [Xtrain[:,k],Xcadac[:,k]]
    multiple_histograms(plt_dat,labels=['train', 'cadac'],bins=50,xlabel=feature_names[k],plt_objs=(fig,ax),normalised=True)

fig, ax = plt.subplots()
multiple_histograms([ytrain, ycadac],labels=['train', 'cadac'],bins=50,xlabel='PCE',plt_objs=(fig,ax),normalised=True)


{'PCE': 0, 'HOMO': 0, 'LUMO': 0, 'ET1': 0, 'ND': 0, 'Acceptor': 'PC71BM', 'DH': 0, 'DL': 0, 'name': 'C1-A1-D1-A1-C1'}


AttributeError: 'AtomsRow' object has no attribute 'homo'

In [ ]:
model_paths = ['weights/lgb_model','weights/PCE_model']
for mp in model_paths:
    print(f'----- {mp} -----')
    model = joblib.load(mp)
    print(type(model))
    pdict = model.get_params()
    for key in pdict.keys():
        print(f'{key} ---> {pdict[key]}')
    print('\n\n')

In [19]:
#!/usr/bin/env python

import numpy as np
from data_utils import split_train_test
from qcnico.plt_utils import histogram
import matplotlib.pyplot as plt
from ase.db import connect
import lightgbm as lgb


def test_model_from_db(Xtrain, ytrain, Xtest, ytest, model_type='test',num_estimators=500):
    # Xtrain, ytrain, Xtest, ytest = split_train_test(db)
    # model = joblib.load(model_params_path)
    # model.set_params(objective='regression', metric='rmse')

    if model_type == 'overfit':
        model = lgb.LGBMRegressor(
            learning_rate=0.5,       # Even more aggressive learning
            max_depth=-1,            # No limit on depth
            num_leaves=2**16,        # A massive number of leaves to force splits
            min_child_samples=1,     # Allow splitting until only 1 sample remains
            n_estimators=5000,       # Huge number of trees to ensure memorization
            reg_alpha=0,             # No L1 regularization
            reg_lambda=0,            # No L2 regularization
            subsample=1.0,           # Use 100% of the data
            colsample_bytree=1.0,    # Use 100% of features
            min_split_gain=0,        # Allow splits even with **no gain**
            importance_type='gain',  # Prioritize gain-based splitting
            force_col_wise=True,     # Helps force more splits
            extra_trees=True,        # Randomizes split selection to allow more splits
            deterministic=True,      # Ensures no randomness
            boosting_type='gbdt',    # Standard boosting
            objective='regression',
            metric='rmse',
            random_state=42
        )

    elif model_type == 'paper':
        model = lgb.LGBMRegressor(
        learning_rate=0.15, 
        max_depth=9,
        # min_child_samples=5,
        n_estimators=39,
        num_leaves = 35,
        random_state=399,
        # reg_alpha=0.2,
        # reg_lambda=0.01,
        objective='regression',
        metric='rmse')

    else:
        model = lgb.LGBMRegressor(
        learning_rate=0.05, 
        max_depth=-1,
        min_child_samples=5,
        n_estimators=num_estimators,
        num_leaves = 2**12,
        random_state=399,
        reg_alpha=0.2,
        reg_lambda=0.01,
        objective='regression',
        metric='rmse')

    model.fit(Xtrain,ytrain)
    ytest_pred = model.predict(Xtest) 
    r = np.corrcoef(ytest, ytest_pred)[0,1]
    return r



path_to_db = 'data/train2.db'
db = connect(path_to_db)

model_sizes = [10,40,100,500]
nmodels = 20

for n in model_sizes:
    print(f'*** num_estimators = {n} ***')
    corr_coefs = np.zeros(nmodels)
    for k in range(nmodels):
        print(k, end = ' ')
        Xtrain, ytrain, Xtest, ytest, _ = split_train_test(db,normalize=True)
        corr_coefs[k] = test_model_from_db(Xtrain,ytrain,Xtest,ytest,model_type='test',num_estimators=n)
    
    fig, ax = plt.subplots()
    fig, ax = histogram(corr_coefs,bins=50,plt_objs=(fig,ax),show=False,usetex=False)
    ax.set_title(f'# of estimators = {n}')
    plt.savefig(f'r_distrib_{n}_estimators.png',dpi=300, bbox_inches="tight")
    plt.show()
        



*** num_estimators = 10 ***
0 [LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000067 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 778
[LightGBM] [Info] Number of data points in the train set: 320, number of used features: 9
[LightGBM] [Info] Start training from score 4.813844
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

<IPython.core.display.Javascript object>

[plt_utils.histogram] dx = 0.005511099720729129


RuntimeError: latex was not able to process the following string:
b'# of estimators = 10'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp_pbubaf8 705d01fc322bfdd55f7e835301f6205f.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./705d01fc322bfdd55f7e835301f6205f.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2022/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2022/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/2022/texmf